#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

Cloning into 'yolov5'...
remote: Enumerating objects: 16531, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 16531 (delta 1), reused 5 (delta 0), pack-reused 16522
Receiving objects: 100% (16531/16531), 15.05 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (11354/11354), done.
/content/yolov5
HEAD is now at 064365d8 Update parse_opt() in export.py to work as in train.py (#10789)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.8 MB/s eta 0:00:00
Setup complete. Using torch 2.2.1+cu121 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


## Step 6: Download a Dataset

Add your Roboflow API key below to download the default money counting dataset. Alternatively, use the code provided by the Roboflow dashboard in the above step to load a custom dataset.

In [ ]:
# Load Dataset

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

names:
- Car
- Vacant
nc: 2
roboflow:
  license: CC BY 4.0
  project: smart_parking
  url: https://universe.roboflow.com/smartparking-tijv5/smart_parking/dataset/2
  version: 2
  workspace: smartparking-tijv5
test: ../test/images
train: SMART_PARKING-2/train/images
val: SMART_PARKING-2/valid/images


# Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 37 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
2024-04-14 06:54:13.972032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 06:54:13.972091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 06:54:13.973904: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 06:54:15.142068: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=./models/custom_yolov5s.yaml, data=/content/yolov5/SMART_PARKING-2/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=37, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, 

# Evaluate Custom YOLOv5 Detector Performance

You can view the training graphs associated with a training job in the `/content/yolov5/runs/train/yolov5s_results/results.png` folder.

Training losses and performance metrics are also saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Locate the directory containing training logs
log_dir = '/content/yolov5/runs/train/yolov5s_results/'

# Get the list of files in the directory
files = os.listdir(log_dir)

# Find the CSV file containing training logs
csv_files = [file for file in files if file.endswith('.csv')]

# Check if any CSV file exists
if len(csv_files) == 0:
    print("No CSV file found in the directory:", log_dir)
else:
    # Assuming there's only one CSV file, load the first one
    csv_file = csv_files[0]
    # Load the training logs
    train_logs = pd.read_csv(os.path.join(log_dir, csv_file))

    # Extract the desired metrics
    metrics = train_logs[['epoch', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss',
                          'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP@0.5',
                          'metrics/mAP@0.5:0.95', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss',
                          'lr0', 'lr1', 'lr2']]

    # Print the metrics
    print(metrics)

    # Plot the training and validation losses
    plt.figure(figsize=(12, 8))
    plt.plot(metrics['epoch'], metrics['train/box_loss'], label='Train Box Loss')
    plt.plot(metrics['epoch'], metrics['train/cls_loss'], label='Train Class Loss')
    plt.plot(metrics['epoch'], metrics['train/dfl_loss'], label='Train DFL Loss')
    plt.plot(metrics['epoch'], metrics['val/box_loss'], label='Validation Box Loss')
    plt.plot(metrics['epoch'], metrics['val/cls_loss'], label='Validation Class Loss')
    plt.plot(metrics['epoch'], metrics['val/dfl_loss'], label='Validation DFL Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot the precision, recall, mAP50, and mAP50-95
    plt.figure(figsize=(12, 8))
    plt.plot(metrics['epoch'], metrics['metrics/precision(B)'], label='Precision')
    plt.plot(metrics['epoch'], metrics['metrics/recall(B)'], label='Recall')
    plt.plot(metrics['epoch'], metrics['metrics/mAP@0.5'], label='mAP@0.5')
    plt.plot(metrics['epoch'], metrics['metrics/mAP@0.5:0.95'], label='mAP@0.5:0.95')
    plt.xlabel('Epoch')
    plt.ylabel('Metrics')
    plt.title('Precision, Recall, mAP@0.5, and mAP@0.5:0.95')
    plt.legend()
    plt.grid(True)
    plt.show()


No CSV file found in the directory: /content/yolov5/runs/train/yolov5s_results/


In [ ]:
import pandas as pd

# Load the training logs
train_logs = pd.read_csv('/content/yolov5/runs/train/yolov5s_results3/results.csv')

# Extract the desired metrics
metrics = train_logs[['epoch', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss',
                      'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP@0.5',
                      'metrics/mAP@0.5:0.95', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss',
                      'lr0', 'lr1', 'lr2']]

# Print the metrics
print(metrics)


KeyError: "None of [Index(['epoch', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss',\n       'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP@0.5',\n       'metrics/mAP@0.5:0.95', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss',\n       'lr0', 'lr1', 'lr2'],\n      dtype='object')] are in the [columns]"